In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [48]:
TABLE_GENERATION_PROMPT = """
Based on the table metadata and schema generate a sql query to answer the user question
MetaData:
TableName: name
Description: Contains the name for each restaurant in the  area of the user
Columns:
id: contains unique id for each restaurant
name: contains name of restaurant

TableName:
TableName: details
Description: Contains all details regarding the restaurants menu
Columns:
restaurant_id: Foreign key that links this table with table `name`
item_type: Categories the item [Starters, Main Course, Desserts]
item_name: Name of item
description: Description regarding the item
ingredients: Ingredients of each item
cost: Cost of each item in $
preparation_time: Time taken to prepare each item in minutes

Schema:
create table restaurants.name(
    id int,
    name VARCHAR(255)
);

create table restaurants.details(
restaurant_id int,
item_type VARCHAR(50),
item_name VARCHAR(255),
description TEXT,
ingredients TEXT,
cost double,
preparation_time int
);

<<Example 1>>
User Query: What are the restaurants in my area

Response:
SELECT * FROM restaurants.name

<<Example 2>>
User Query: What is the menu for the restaurant Bistro Fusion?
    
Response:
SELECT * FROM restaurants.details
WHERE restaurant_id in (SELECT id FROM name WHERE lower(restaurants.name) like "%bistro fusion%")

<<Example 3>>
User Query: {input}

Response:
"""

In [49]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613")

In [50]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are skilled SQL Engineer who can extract relevant entities from a question and convert that into a SQL query"),
    ("user", "{input}")
])

In [51]:
output_parser = StrOutputParser()

In [55]:
# question = "What are there for starters in The Culinary Haven"
question = TABLE_GENERATION_PROMPT.format(input="Could you tell me dishes that get ready within 10 minutes and are less than 5$ and contains Chicken")

In [56]:
chain = prompt | llm | output_parser

In [57]:
output = chain.invoke({"input": question})
print(output)

SELECT * FROM restaurants.details
WHERE preparation_time <= 10 AND cost < 5 AND ingredients LIKE "%chicken%"


In [ ]:
[Chicken, Beef, mutton, fish] 
Chicken : 2
Beef: 3
mutton : 0
fish: 1